In [1]:
!pip3 install cleantext
!pip3 install dateparser

     |████████████████████████████████| 286kB 13.9MB/s 


In [2]:
import re
import string
from cleantext import clean
import requests
import json
import glob
import os
import pandas as pd
import numpy as np
import dateparser
import string 
#Data cleaning 
import bs4 as bs
import urllib.request
import re
import nltk

In [3]:
from nltk.corpus import stopwords
import nltk
import nltk
nltk.download('punkt')
nltk.download("stopwords")
chachedWords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/MyDrive/Theolex/theolex_document_processing/

/content/drive/MyDrive/Theolex/theolex_document_processing


In [6]:
def clean(file):
    article_text = re.sub(r'\[[0-9]*\]', ' ', file)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    return  article_text,formatted_article_text


In [7]:
text_path = "7100-15/cftc_text_7100-15_press-release.txt"
def load_data(text_path):
    with open(text_path) as f:
        file = f.read()
    return file
file=load_data(text_path)
doc,format=clean(file)

In [8]:
def unicode(text):
    text_unicode = doc 
    # encoding the text to ASCII format
    text_encode = text_unicode.encode(encoding="ascii", errors="ignore")
    # decoding the text
    text_decode = text_encode.decode()
    # cleaning the text to remove extra whitespace 
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

In [9]:
import nltk
text=unicode(file)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
sentence_list = nltk.sent_tokenize(text)


In [11]:
#cleaned_text = ' '.join(sentence_list)


In [12]:
def is_section_num(text):
    
    return re.search(r'^[MDCLXVIAB]+\.$',text) is not None

    
def ends_with_ponc(text):
    return text[-1] in punctuation if text else False 

def start_with_upper(text):
    return re.match('^[A-Z]', text) is not None

def is_title(text):
    uppers = [word.isupper() for word in text.split(" ")]
    return sum(uppers) / len(uppers) > 0.6
is_title("UNITED STATES OF AMERICA Before the COMMODITY FUTURES TRADING COMMISSION")



True

In [13]:
import nltk
from nltk.tokenize import sent_tokenize
t = sent_tokenize("Three days later, the SEC  filed a federal injunctive action against Sentinel, SEC v.  Sentinel,  et al., No.  07 CV 4684 (N.D. Ill. filed Aug. 20, 2007), and on April 28, 2008, the CFTC filed a Complaint against Sentinel, Bloom and Sentinel's Senior Vice President and head trader, Charles K. Mosley, seeking injunctive and other equitable relief, as well as the imposition of civil penalties, for violating various provisions of the Act and Commission Regulations.")

In [14]:
def same_sentence(sent1, sent2):
    #print(sent1,"\n",sent2)
    
    # sent empty
    if (not sent1) or (not sent1['text']) :
        return True
    # section number
    if sent1['is_section_num']:
        return True
    
    # very short sentence
    if len(sent1['text']) < 50:
        return False
    
    # ponctuation
    if sent1['ends_with_ponc']:
        return False
    
    if sent1['is_title']:
        return False

    if sent2['is_title']:
        return False
    
    return True

In [15]:
punctuation=['.','?','!']

In [16]:
def clean_doc(file_text):
    pages=[]
    for page in file_text.split("\x0c"):
        # clen text
        page_meta = [{'text': clean(para)} 
                            for para in page.split('\n')]
        clean_page = []
        previous_line = {}
        text= ""
        # add meta data
        for line in page_meta:
            line['is_section_num'] = is_section_num(str(line['text']))
            line['is_title'] = is_title(str(line['text']))
            line['ends_with_ponc'] = ends_with_ponc(str(line['text']))
            line['is_alpha'] = sum(c.isalpha() for c in str(line['text']))
            line['start_with_upper']  = start_with_upper(str(line['text']))
                                
            # not relevant line
            if not line['is_alpha']:
                continue
            
            if not same_sentence(previous_line, line):
                if text:
                    clean_page.append(text)
                    text = ""
                
            previous_line = line
            text = " ".join([text, str(line['text'])])
        if len(clean_page):
            pages.append(clean_page)
    return pages

In [19]:

def word_frequency(text):
    word_frequencies = {}
    for word in nltk.word_tokenize(text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    return word_frequencies

In [20]:
def sentence_score(sentence_list,word_frequencies):
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    return sentence_scores


In [21]:
import heapq
def summary(sentence_scores):
    summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)

    
    return summary

In [22]:
text_path = "./7100-15/cftc_text_7100-15_order-allied-markets-llc-et-al.txt"
with open(text_path) as f:
    file = f.read()
article_text, formatted_article_text=clean(file)

sentence_list = nltk.sent_tokenize(article_text)
# word_frequencies=word_frequency(formatted_article_text)
# stopwords = nltk.corpus.stopwords.words('english')

# maximum_frequncy = max(word_frequencies.values())

# for word in word_frequencies.keys():
#     word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
# sentence_scores = sentence_score(sentence_list, word_frequencies)
    
# summary(sentence_scores)



##  Training textsplit model using word vector

In [23]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline


In [24]:
!ls 7100-15

cftc_text_7100-15_complaint-allied-markets-llc-et-al.txt
cftc_text_7100-15_order-allied-markets-llc-et-al.txt
cftc_text_7100-15_press-release.txt


In [25]:
corpus_path = './text8'  # be sure your corpus is cleaned from punctuation and lowercased
if not os.path.exists(corpus_path):
    !wget http://mattmahoney.net/dc/text8.zip
    !unzip {corpus_path}

In [26]:
links = './7100-15/cftc_text_7100-15_complaint-allied-markets-llc-et-al.txt'

## No case/docket

In [27]:
clean_page = clean_doc(file)
clean_page[1][2:4]

[" ('The Court possesses jurisdiction over the parties and over the subject matter ', 'The Court possesses jurisdiction over the parties and over the subject matter ')",
 " ('of this action pursuant to Section 6c of the CEA, 7 U.S.C. § 13a- l (2012). ', 'of this action pursuant to Section c of the CEA U S C a l ')"]

In [28]:
def get_case(first_page):
    for line in first_page:
        if len(line) < 40:
            p = re.compile("NO[\.:]\s*.+")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()
            
    for line in first_page:   
        if len(line) < 40:    
            p = re.compile("\d*-?CV-\d+.*")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()

In [29]:
get_case([' CFTC Docket No. SD 20-01'])

'NO. SD 20-01'

## Defendant

In [30]:
!pip3 install spacy

In [31]:
!python3 -m spacy download en_core_web_sm


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [32]:
import spacy
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")
# Process whole documents
text = ("\n".join(clean_page[0]))
print(text)
doc = nlp(text)

for entity in doc.ents:
    if entity.label_ == "ORG":
        print(entity.text,":       ", entity.label_)

 ('UNITED STATES DISTRICT COURT ', 'UNITED STATES DISTRICT COURT ')
 ('MIDDLE DISTRICT OF FLORIDA ', 'MIDDLE DISTRICT OF FLORIDA ')
 ('Jacksonville Division ', 'Jacksonville Division ')
 ('v. ', 'v ')
 ('Case No. 3:15-cv-5-J-34MCR ', 'Case No cv J MCR ')
 ('U.S. COMMODITY FUTURES ', 'U S COMMODITY FUTURES ')
 ('TRADING COMMISSION, ', 'TRADING COMMISSION ')
 ('Plaintiff, ', 'Plaintiff ')
 ('ALLIED MARKETS LLC, ', 'ALLIED MARKETS LLC ')
 ('JOSHUA GILLILAND, and ', 'JOSHUA GILLILAND and ')
 ('CHAWALIT WONGKHIAO, ', 'CHAWALIT WONGKHIAO ')
 ('Defendants. ', 'Defendants ')
 ("ORDER GRANTING PLAINTIFF'S EX PARTE ", 'ORDER GRANTING PLAINTIFF S EX PARTE ')
 ('MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING ', 'MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING ')
 ('PRELIMINARY fNJUNCTION HEARING ', 'PRELIMINARY fNJUNCTION HEARING ')
 ('THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory ', 'THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statuto

In [33]:
print("\n".join(clean_page[0]))

 ('UNITED STATES DISTRICT COURT ', 'UNITED STATES DISTRICT COURT ')
 ('MIDDLE DISTRICT OF FLORIDA ', 'MIDDLE DISTRICT OF FLORIDA ')
 ('Jacksonville Division ', 'Jacksonville Division ')
 ('v. ', 'v ')
 ('Case No. 3:15-cv-5-J-34MCR ', 'Case No cv J MCR ')
 ('U.S. COMMODITY FUTURES ', 'U S COMMODITY FUTURES ')
 ('TRADING COMMISSION, ', 'TRADING COMMISSION ')
 ('Plaintiff, ', 'Plaintiff ')
 ('ALLIED MARKETS LLC, ', 'ALLIED MARKETS LLC ')
 ('JOSHUA GILLILAND, and ', 'JOSHUA GILLILAND and ')
 ('CHAWALIT WONGKHIAO, ', 'CHAWALIT WONGKHIAO ')
 ('Defendants. ', 'Defendants ')
 ("ORDER GRANTING PLAINTIFF'S EX PARTE ", 'ORDER GRANTING PLAINTIFF S EX PARTE ')
 ('MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING ', 'MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING ')
 ('PRELIMINARY fNJUNCTION HEARING ', 'PRELIMINARY fNJUNCTION HEARING ')
 ('THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory ', 'THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statuto

In [34]:
!pip3 install transformers
#!pip3 install torch torchvision torchaudio
#!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

In [35]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Question answering pipeline, specifying the checkpoint identifier
nlp = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')

In [36]:
def clean_spec_chars(text: str) -> tuple:
    """first text cleaning based on regex, just keep text not spec chars
    return tupple of text"""

    # article text
    article_text = re.sub(r"\[[0-9]*\]", " ", text)
    article_text = re.sub(r"\s+", " ", article_text)

    # formated text
    formatted_article_text = re.sub("[^a-zA-Z]", " ", article_text)
    formatted_article_text = re.sub(r"\s+", " ", formatted_article_text)

    return article_text, formatted_article_text

In [37]:
def get_token(text):
    sentence_list = nltk.sent_tokenize(text)
    sentence_list
    return sentence_list

In [38]:
data, _=clean_spec_chars(file)
print(data)
list_=get_token(data)
print(len(list_))

UNITED STATES DISTRICT COURT MIDDLE DISTRICT OF FLORIDA Jacksonville Division v. Case No. 3:15-cv-5-J-34MCR U.S. COMMODITY FUTURES TRADING COMMISSION, Plaintiff, ALLIED MARKETS LLC, JOSHUA GILLILAND, and CHAWALIT WONGKHIAO, Defendants. ORDER GRANTING PLAINTIFF'S EX PARTE MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING PRELIMINARY fNJUNCTION HEARING THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory Restraining Order and Motion for Preliminary Injunction (Doc. S-4; Motion), filed on January 5, 2015. Simultaneously with the Motion, the United States Commodity Futures Trading Commission (Plaintiff or CFTC) filed a Complaint for Injunctive Relief, Civil Monetary Penalty, and Other Equitable Relief (Doc. S-3) against Defendants Allied Markets LLC, Joshua Gilliland, and Chawalit Wongkhiao (collectively, Defendants). In the Motion, the CFTC moves, pursuant to Section 6c(a) of the Commodity Exchange Act (CEA), 7 U.S.C. § 13a-l (a) (2012), for an ex parte statutory

In [39]:
def get_section_indx(list_token):
    idx=[]
    for i in range(len(list_token)):
        if is_title(list_token[i]):
            idx.append(i)
            
    return idx

def get_structure(text):     
    list_token=get_token(text)
    idx=get_section_indx(list_token)
    j=0
    structure=[]
    k=0
    for i in idx:
        section={}
        section['content']=list_token[j:i+1]
        section['header']=list_token[i]
        section['id']=k
        structure.append(section)
        j=i+1
        k=k+1
        
    return structure

In [40]:
section=get_structure(data)
section

[{'content': ['UNITED STATES DISTRICT COURT MIDDLE DISTRICT OF FLORIDA Jacksonville Division v. Case No.'],
  'header': 'UNITED STATES DISTRICT COURT MIDDLE DISTRICT OF FLORIDA Jacksonville Division v. Case No.',
  'id': 0},
 {'content': ['3:15-cv-5-J-34MCR U.S. COMMODITY FUTURES TRADING COMMISSION, Plaintiff, ALLIED MARKETS LLC, JOSHUA GILLILAND, and CHAWALIT WONGKHIAO, Defendants.'],
  'header': '3:15-cv-5-J-34MCR U.S. COMMODITY FUTURES TRADING COMMISSION, Plaintiff, ALLIED MARKETS LLC, JOSHUA GILLILAND, and CHAWALIT WONGKHIAO, Defendants.',
  'id': 1},
 {'content': ["ORDER GRANTING PLAINTIFF'S EX PARTE MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING PRELIMINARY fNJUNCTION HEARING THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory Restraining Order and Motion for Preliminary Injunction (Doc.",
   'S-4; Motion), filed on January 5, 2015.',
   'Simultaneously with the Motion, the United States Commodity Futures Trading Commission (Plaintiff or CFTC) filed 

In [41]:
def get_case(page):
    for line in page[1]['content']:
        
        print(line)
        if len(line) < 50:
            p = re.compile("NO[\.:]\s*.+")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()
            
    for line in  page[0]['content']:   
        if len(line) < 50:    
            p = re.compile("\d*-?CV-\d+.*")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()

In [95]:
section[2]

{'content': ["ORDER GRANTING PLAINTIFF'S EX PARTE MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING PRELIMINARY fNJUNCTION HEARING THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory Restraining Order and Motion for Preliminary Injunction (Doc.",
  'S-4; Motion), filed on January 5, 2015.',
  'Simultaneously with the Motion, the United States Commodity Futures Trading Commission (Plaintiff or CFTC) filed a Complaint for Injunctive Relief, Civil Monetary Penalty, and Other Equitable Relief (Doc.',
  'S-3) against Defendants Allied Markets LLC, Joshua Gilliland, and Chawalit Wongkhiao (collectively, Defendants).',
  'In the Motion, the CFTC moves, pursuant to Section 6c(a) of the Commodity Exchange Act (CEA), 7 U.S.C.',
  '§ 13a-l (a) (2012), for an ex parte statutory restraining order freezing assets and prohibiting the destruction of books, records, or other documents; and for an order requiring Defendants to show cause why a preliminary injunction should not

In [67]:
import spacy
def get_entities(section):
    entities=[]
    # Load English tokenizer, tagger, parser and NER
    nlp = spacy.load("en_core_web_sm")
    # Process whole documents
    for text in section:
        text = ("\n".join(section[2]['content']))
        doc = nlp(text)
        # Find named entities, phrases and concepts
        for entity in doc.ents:
            if entity.label_ == "ORG":
                entities.append([entity.text,":       ", entity.label_])
            
    return entities

In [87]:
def get_header(section):
  sec0="".join(section[0]['content'])
  sec1="".join(section[1]['content'])
  entete=[sec0]
  entete.append(sec1)
  header="".join(entete)
  return header


In [82]:
header=create_header(section)
header

'UNITED STATES DISTRICT COURT MIDDLE DISTRICT OF FLORIDA Jacksonville Division v. Case No.3:15-cv-5-J-34MCR U.S. COMMODITY FUTURES TRADING COMMISSION, Plaintiff, ALLIED MARKETS LLC, JOSHUA GILLILAND, and CHAWALIT WONGKHIAO, Defendants.'

In [89]:
print(nlp(question="what is the case No?", context=header, topk=4))

[{'score': 0.4023226201534271, 'start': 89, 'end': 106, 'answer': '3:15-cv-5-J-34MCR'}, {'score': 0.049178872257471085, 'start': 91, 'end': 106, 'answer': '15-cv-5-J-34MCR'}, {'score': 0.04214375838637352, 'start': 89, 'end': 90, 'answer': '3'}, {'score': 0.009276355616748333, 'start': 81, 'end': 90, 'answer': 'Case No.3'}]


In [93]:
print(nlp(question="Who is  Defendants", context=header, topk=3))

[{'score': 0.6581836342811584, 'start': 203, 'end': 221, 'answer': 'CHAWALIT WONGKHIAO'}, {'score': 0.019663704559206963, 'start': 203, 'end': 211, 'answer': 'CHAWALIT'}, {'score': 0.006906569469720125, 'start': 223, 'end': 233, 'answer': 'Defendants'}]


In [88]:
def get_defendant(section):
    nlp = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')
    defendant=nlp(question="Who is Defendant?", context=get_header(section),topk=3)
    return defendant

In [ ]:
defendant=get_defendant(section)
list_defend=[]
for defend in defendant:
  if defend['score']> 0.5:
    list_defend.append(defend['answer'])

In [ ]:
list_defend

['Joshua Gilliland and Chawalit Wongkhiao', 'Allied Markets LLC']

# STOP HERE

In [ ]:
section[2]['content']



['3:15-cv-5-J-34MCR U.S. COMMODITY FUTURES TRADING COMMISSION, Plaintiff, ALLIED MARKETS LLC, JOSHUA GILLILAND, and CHAWALIT WONGKHIAO, Defendants.',
 "ORDER GRANTING PLAINTIFF'S EX PARTE MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING PRELIMINARY fNJUNCTION HEARING THIS CAUSE is before the Court on Plaintiffs Ex Parte Motion for Statutory Restraining Order and Motion for Preliminary Injunction (Doc.",
 'S-4; Motion), filed on January 5, 2015.',
 'Simultaneously with the Motion, the United States Commodity Futures Trading Commission (Plaintiff or CFTC) filed a Complaint for Injunctive Relief, Civil Monetary Penalty, and Other Equitable Relief (Doc.',
 'S-3) against Defendants Allied Markets LLC, Joshua Gilliland, and Chawalit Wongkhiao (collectively, Defendants).',
 'In the Motion, the CFTC moves, pursuant to Section 6c(a) of the Commodity Exchange Act (CEA), 7 U.S.C.',
 '§ 13a-l (a) (2012), for an ex parte statutory restraining order freezing assets and prohibiting the destructi

## Extracted authorities

In [ ]:
UNITED STATES DISTRICT COURT
FOR THE MIDDLE DISTRICT OF FLORIDA 

## Code law violation

In [ ]:
violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C.
§ 6c(a)(5)(C) (2018).

In [ ]:
Section 4c(a)(5) of the Act, 7 U.S.C.
§ 6c(a)(5) (2012).

## Violation period

In [ ]:
line = 'The Commodity Futures Trading Commission ("Commission") has reason to believe that from at least July 2012 through March 2017 ("Relevant Period"), Propex Derivatives Pty Ltd. ("Propex") violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C. 6c(a)(5)(C) (2018). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Propex engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.'

In [ ]:
("Relevant Period")

## Transaction amount

In [ ]:
pour complaint tous les montants vers Transaction amounts

## Defendant

In [ ]:
Propex Derivatives Pty Ltd, Respondent.

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm

## Nature of violations

In [ ]:
II. FINDINGS
The Commission finds the following:
A. SUMMARY
During the Relevant Period, Propex, by and through a Propex trader ("Trader A"),·
engaged in thousands of instances of the disruptive trading practice known as "spoofing"
(bidding or offering with the intent to cancel the bid or offer before execution) in the E-mini
S&P 500 futures contracts traded on the Chicago Mercantile Exchange ("CME"), a futures
exchange and designated contract market which is owned and operated by CME Group Inc. This
conduct violated Section 4c(a)(5)(C) of the Act, 7 U.S.C. § 6c(a)(5)(C) (2018). 

In [ ]:
cftc_full_list = pd.read_excel("cftc_full_list.xlsx")

In [ ]:
def gen_line_rep(rep):
    original_path = f"./cftc/original/{rep}"
    text_path = f"./cftc/text/{rep}"
    meta_path = f"./cftc/meta-data/{rep}"
    lines = []
    for file in glob.glob(f"{meta_path}/*.json"):
        filename = os.path.basename(file)

        with open(file) as json_file: 
            data = json.load(json_file)     

        data['filename'] = filename.replace(".json","")
        data['folder'] = rep
        with open(f"{text_path}/{filename.replace('.json','.txt')}") as f:
            data['doc_text'] = f.read()  


        lines.append(data)
    return lines

In [ ]:
meta_data = []
for index, row in cftc_full_list[~cftc_full_list.scraped_folder.isnull()].iterrows():
    try:
        meta_data.extend(gen_line_rep(row['scraped_folder']))
    except:
        print("error on",  row['scraped_folder'])
len(meta_data)

error on 7882-19
error on 8262-20
error on 8239-20


756

In [ ]:
df_meta_data = pd.DataFrame(meta_data)

In [ ]:
df_meta_data['doc_clean'] = df_meta_data.doc_text.apply(clean_doc)

In [ ]:
df_meta_data['first_page'] = df_meta_data['doc_clean'].str[0]

In [ ]:
df_meta_data = df_meta_data[~df_meta_data['first_page'].isnull()]

## Get reference

In [ ]:
df_meta_data['reference'] = df_meta_data.first_page.apply(get_case)

## Get defendant

In [ ]:
first_page = [text for text in df_meta_data.first_page.values[3] if len(text) > 100]

In [ ]:
first_page

[' The Commodity Futures Trading Commission ("Commission") has reason to believe that from in or about August 2012 to at least January 2015 (the "Relevant Period"), Respondents Sharpe Signa, LLC ("Sharpe") and Garen Ovsepyan ("Ovsepyan") violated Sections 2(c)(2)(C)(iii)(I)(bb), 4b(a)(2)(A) and (C), 4o(l)(A) and (B), and 6(c)(2) of the Act, 7 U.S.C.',
 ' SSSS 2(c)(2)(C)(iii)(I)(bb), 6b(a)(2)(A) and (C), 6o(l)(A) and (B), and 9(2) (2012), and Commission Regulation ("Regulation") 5.3(a)(3)(i), 17 C.F.R. SS 5.3(a)(3)(i) (2014), and Respondent Haeres Capital, LLC ("Haeres") and Ovsepyan violated Section 6(c )(2) of the Act, 7 U.S.C. SS 9(2) (2012). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Respondent(s) engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.',
 ' In anticipation of the institution of a

In [ ]:
print(nlp(question="Who violeted?", context=".".join(first_page)))

{'score': 0.1322660595178604, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}


In [ ]:
print(nlp(question="Who is the defendant?", context=".".join(first_page), topk=3))

[{'score': 0.009618670679628849, 'start': 208, 'end': 222, 'answer': 'Garen Ovsepyan'}, {'score': 0.0038621597923338413, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}, {'score': 0.0027537851128727198, 'start': 175, 'end': 203, 'answer': 'Sharpe Signa, LLC ("Sharpe")'}]


In [ ]:
#df_meta_data[['reference', 'folder', 'filename']][df_meta_data.reference.isnull()]

## Type

In [ ]:
df_meta_data['is_order'] = df_meta_data.filename.str.contains("order")

In [ ]:
df_meta_data['is_complaint'] = df_meta_data.filename.str.contains("complaint")

In [ ]:
df_meta_data['type'] = np.where(df_meta_data['is_order'], "Order CFTC", 
         np.where(df_meta_data['is_complaint'], 
                  "Complaint CFTC", None))